In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import Dataset, ConcatDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_grey = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Grayscale(num_output_channels=3)])

batch_size = 6

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transform)

trainset_grey = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transform_grey)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
input1=[]
label=[]
for i, data in enumerate(trainset):
    inputs, labels = data
    input1.append(inputs)
    label.append(labels)

# Define your condition (e.g., labels above a certain threshold)
threshold = 4
condition_indices = [i for i, j in enumerate(label) if j > threshold]
non_condition_indices = [i for i, j in enumerate(label) if j <= threshold]

GREYSET = torch.utils.data.Subset(trainset_grey, condition_indices)

COLORSET = torch.utils.data.Subset(trainset, non_condition_indices)


In [4]:
FINAL = ConcatDataset([GREYSET, COLORSET])


trainloader = torch.utils.data.DataLoader(FINAL, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

trainloader2 = torch.utils.data.DataLoader(trainset_grey, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [8]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))

In [9]:
for epoch in range(10):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


        inputs = inputs.to(device)
        labels = labels.to(device)


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

0
[1,  2000] loss: 1.464
[1,  4000] loss: 1.117
[1,  6000] loss: 1.015
[1,  8000] loss: 0.969
1
[2,  2000] loss: 0.899
[2,  4000] loss: 0.893
[2,  6000] loss: 0.854
[2,  8000] loss: 0.863
2
[3,  2000] loss: 0.796
[3,  4000] loss: 0.795
[3,  6000] loss: 0.799
[3,  8000] loss: 0.786
3
[4,  2000] loss: 0.732
[4,  4000] loss: 0.755
[4,  6000] loss: 0.757
[4,  8000] loss: 0.746
4
[5,  2000] loss: 0.698
[5,  4000] loss: 0.704
[5,  6000] loss: 0.718
[5,  8000] loss: 0.730
5
[6,  2000] loss: 0.676
[6,  4000] loss: 0.676
[6,  6000] loss: 0.707
[6,  8000] loss: 0.688
6
[7,  2000] loss: 0.647
[7,  4000] loss: 0.663
[7,  6000] loss: 0.668
[7,  8000] loss: 0.675
7
[8,  2000] loss: 0.621
[8,  4000] loss: 0.622
[8,  6000] loss: 0.651
[8,  8000] loss: 0.658
8
[9,  2000] loss: 0.611
[9,  4000] loss: 0.614
[9,  6000] loss: 0.623
[9,  8000] loss: 0.634
9
[10,  2000] loss: 0.572
[10,  4000] loss: 0.610
[10,  6000] loss: 0.597
[10,  8000] loss: 0.623
Finished Training


In [10]:
total = 0 
correct =0

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Files already downloaded and verified
Accuracy of the network on the 10000 test images: 35 %


In [17]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))


for epoch in range(10):  # loop over the dataset multiple times
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader2, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


        inputs = inputs.to(device)
        labels = labels.to(device)


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

0
[1,  2000] loss: 1.871
[1,  4000] loss: 1.636
[1,  6000] loss: 1.509
[1,  8000] loss: 1.436
1
[2,  2000] loss: 1.345
[2,  4000] loss: 1.311
[2,  6000] loss: 1.276
[2,  8000] loss: 1.245
2
[3,  2000] loss: 1.175
[3,  4000] loss: 1.162
[3,  6000] loss: 1.178
[3,  8000] loss: 1.137
3
[4,  2000] loss: 1.069
[4,  4000] loss: 1.078
[4,  6000] loss: 1.073
[4,  8000] loss: 1.084
4
[5,  2000] loss: 1.007
[5,  4000] loss: 1.014
[5,  6000] loss: 1.016
[5,  8000] loss: 1.038
5
[6,  2000] loss: 0.944
[6,  4000] loss: 0.995
[6,  6000] loss: 0.969
[6,  8000] loss: 1.000
6
[7,  2000] loss: 0.922
[7,  4000] loss: 0.930
[7,  6000] loss: 0.969
[7,  8000] loss: 0.951
7
[8,  2000] loss: 0.871
[8,  4000] loss: 0.901
[8,  6000] loss: 0.936
[8,  8000] loss: 0.929
8
[9,  2000] loss: 0.845
[9,  4000] loss: 0.875
[9,  6000] loss: 0.901
[9,  8000] loss: 0.911
9
[10,  2000] loss: 0.818
[10,  4000] loss: 0.859
[10,  6000] loss: 0.855
[10,  8000] loss: 0.882
Finished Training


In [18]:
total = 0 
correct =0

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Files already downloaded and verified
Accuracy of the network on the 10000 test images: 60 %
